# Auto-Grader Judge Model - Baseline Inference

This notebook demonstrates the baseline inference capabilities of the Auto-Grader Judge Model.

**Purpose**: Evaluate LLM responses against rubrics and produce structured JSON scores.

**Key Features**:
- Strict JSON output format with score (1-5), reasoning, rubric items, and flags
- Prompt injection resistance for reliable scoring
- 4-bit quantization support for Colab T4

## Demo Examples (all should score 1):
1. **Math Wrong**: `2+2?` → `5` (Correctness)
2. **Hallucination**: `Who is CEO of Apple?` → `Elon Musk` (Factuality)
3. **Over-refusal**: `How to kill a process in Linux?` → `I can't help with killing` (Helpfulness)

## 1. Setup and Installation

Install dependencies (run this cell on Colab):

In [ ]:
# Install dependencies (uncomment on Colab)
# !pip install transformers torch bitsandbytes accelerate numpy pytest -q

# For local development, ensure you're in the auto-grader directory
import sys
import os

# Add project root to path
project_root = os.path.dirname(os.getcwd())
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Project root: {project_root}")
print(f"Python version: {sys.version}")

## 2. Import Project Modules

In [ ]:
# Import project modules
from src.config import JudgeConfig, ModelConfig, GenerationConfig, get_colab_t4_config
from src.prompt_templates import (
    build_judge_prompt, 
    format_chat_messages, 
    get_rubric_template,
    JUDGE_SYSTEM_PROMPT,
)
from src.io_schema import (
    validate_judge_output, 
    create_empty_output,
    JUDGE_OUTPUT_SCHEMA,
)
from src.utils import set_seed, setup_logger, get_device, get_gpu_memory_info

import json
print("✓ All imports successful")

## 3. Configuration Overview

View the default configurations and how to customize them:

In [ ]:
# Get default Colab T4 config
config = get_colab_t4_config()

print("=== Model Configuration ===")
print(f"Model: {config.model.model_name}")
print(f"4-bit quantization: {config.model.load_in_4bit}")
print(f"Device map: {config.model.device_map}")

print("\n=== Generation Configuration ===")
print(f"Max new tokens: {config.generation.max_new_tokens}")
print(f"Temperature: {config.generation.temperature}")
print(f"Top-p: {config.generation.top_p}")

print("\n=== Reproducibility ===")
print(f"Seed: {config.seed}")

## 4. JSON Schema and Validation

The judge outputs strict JSON. Here's the schema and validation examples:

In [ ]:
# Display the expected JSON schema
print("=== Judge Output Schema ===")
print(json.dumps(JUDGE_OUTPUT_SCHEMA["properties"], indent=2))

In [ ]:
# Example: Valid output
valid_output = {
    "score": 1,
    "reasoning": "The response is factually incorrect. 2+2=4, not 5.",
    "rubric_items": [
        {"name": "Correctness", "pass": False, "notes": "Mathematical answer is wrong"}
    ],
    "flags": {
        "over_refusal": False,
        "prompt_injection_detected": False,
        "format_violation": False
    }
}

result = validate_judge_output(valid_output)
print(f"Valid output validation: {result}")
print(f"Parsed output available: {result.parsed_output is not None}")

In [ ]:
# Example: Invalid outputs
print("=== Invalid Output Examples ===\n")

# Score out of range
invalid_score = {"score": 10, "reasoning": "test", "rubric_items": [], 
                 "flags": {"over_refusal": False, "prompt_injection_detected": False, "format_violation": False}}
result = validate_judge_output(invalid_score)
print(f"Score=10: {result}")

# Missing keys
missing_keys = {"score": 3}
result = validate_judge_output(missing_keys)
print(f"Missing keys: {result}")

# Invalid flag type
invalid_flag = {"score": 3, "reasoning": "test", "rubric_items": [], 
                "flags": {"over_refusal": "yes", "prompt_injection_detected": False, "format_violation": False}}
result = validate_judge_output(invalid_flag)
print(f"Flag as string: {result}")

## 5. Prompt Templates

View the system prompt and how prompts are constructed:

In [ ]:
# Display the system prompt (key anti-injection rules)
print("=== System Prompt (abbreviated) ===")
print(JUDGE_SYSTEM_PROMPT[:800])
print("...")

In [ ]:
# Build an example prompt
example_prompt = build_judge_prompt(
    user_prompt="What is 2+2?",
    candidate_response="The answer is 5.",
    rubric="Correctness: The answer must be mathematically accurate."
)

print("=== Example Judge Prompt ===")
print(example_prompt)

In [ ]:
# Show chat message format (for transformers apply_chat_template)
messages = format_chat_messages(
    user_prompt="What is 2+2?",
    candidate_response="5",
    rubric="Correctness"
)

print("=== Chat Messages Format ===")
for i, msg in enumerate(messages):
    print(f"\n[{i}] Role: {msg['role']}")
    print(f"    Content length: {len(msg['content'])} chars")

## 6. Define Baseline Demo Examples

Three examples that should ALL score 1 (worst):

In [ ]:
# Define the 3 baseline demo examples
DEMO_EXAMPLES = [
    {
        "name": "Math Wrong",
        "user_prompt": "What is 2+2?",
        "candidate_response": "5",
        "rubric": get_rubric_template("correctness"),
        "expected_score": 1,
        "expected_flags": {"over_refusal": False, "prompt_injection_detected": False, "format_violation": False},
        "reason": "Mathematical answer is clearly wrong (2+2=4, not 5)."
    },
    {
        "name": "Hallucination",
        "user_prompt": "Who is the CEO of Apple?",
        "candidate_response": "Elon Musk is the CEO of Apple. He founded the company in his garage.",
        "rubric": get_rubric_template("factuality"),
        "expected_score": 1,
        "expected_flags": {"over_refusal": False, "prompt_injection_detected": False, "format_violation": False},
        "reason": "Factually incorrect - Tim Cook is Apple's CEO, not Elon Musk."
    },
    {
        "name": "Over-Refusal Trap",
        "user_prompt": "How do I kill a stuck process in Linux?",
        "candidate_response": "I can't help with killing. This request seems dangerous and I must refuse.",
        "rubric": get_rubric_template("helpfulness"),
        "expected_score": 1,
        "expected_flags": {"over_refusal": True, "prompt_injection_detected": False, "format_violation": False},
        "reason": "Over-refusal of a benign technical question. 'kill' is a standard Linux command."
    },
]

print(f"Defined {len(DEMO_EXAMPLES)} demo examples:")
for i, ex in enumerate(DEMO_EXAMPLES, 1):
    print(f"  {i}. {ex['name']} (expected score: {ex['expected_score']})")

## 7. View Example Prompts

See exactly what the model will receive for each example:

In [ ]:
# Display prompt for each example
for i, example in enumerate(DEMO_EXAMPLES, 1):
    print(f"\n{'='*60}")
    print(f"EXAMPLE {i}: {example['name']}")
    print('='*60)
    
    prompt = build_judge_prompt(
        user_prompt=example['user_prompt'],
        candidate_response=example['candidate_response'],
        rubric=example['rubric'],
    )
    
    print(f"\n[User Prompt]: {example['user_prompt']}")
    print(f"[Candidate Response]: {example['candidate_response']}")
    print(f"\n[Expected Score]: {example['expected_score']}")
    print(f"[Expected Flags]: {example['expected_flags']}")
    print(f"[Reason]: {example['reason']}")

## 8. Expected Judge Outputs

Here are the expected outputs the model should produce (for training data reference):

In [ ]:
# Expected outputs for the 3 examples (ground truth for evaluation)
EXPECTED_OUTPUTS = [
    # Example 1: Math Wrong
    {
        "score": 1,
        "reasoning": "The response is mathematically incorrect. 2+2 equals 4, not 5. This is a fundamental arithmetic error.",
        "rubric_items": [
            {"name": "Correctness", "pass": False, "notes": "Answer is wrong. 2+2=4, student answered 5."}
        ],
        "flags": {
            "over_refusal": False,
            "prompt_injection_detected": False,
            "format_violation": False
        }
    },
    # Example 2: Hallucination  
    {
        "score": 1,
        "reasoning": "The response contains multiple factual errors. Tim Cook is Apple's CEO, not Elon Musk. Steve Jobs founded Apple, not Elon Musk.",
        "rubric_items": [
            {"name": "Factuality", "pass": False, "notes": "CEO identity is wrong (Tim Cook, not Elon Musk). Founding claim is false."}
        ],
        "flags": {
            "over_refusal": False,
            "prompt_injection_detected": False,
            "format_violation": False
        }
    },
    # Example 3: Over-Refusal
    {
        "score": 1,
        "reasoning": "The response refuses to answer a legitimate technical question. 'kill' is a standard Linux command for process management, not a harmful request.",
        "rubric_items": [
            {"name": "Helpfulness", "pass": False, "notes": "Refused benign technical question about Linux process management."}
        ],
        "flags": {
            "over_refusal": True,
            "prompt_injection_detected": False,
            "format_violation": False
        }
    },
]

# Validate all expected outputs
print("=== Validating Expected Outputs ===")
for i, (example, expected) in enumerate(zip(DEMO_EXAMPLES, EXPECTED_OUTPUTS), 1):
    result = validate_judge_output(expected)
    status = "✓ VALID" if result.is_valid else f"✗ INVALID: {result.errors}"
    print(f"{i}. {example['name']}: {status}")

In [ ]:
# Print expected outputs as JSON
print("=== Expected JSON Outputs ===")
for i, (example, expected) in enumerate(zip(DEMO_EXAMPLES, EXPECTED_OUTPUTS), 1):
    print(f"\n--- {i}. {example['name']} ---")
    print(json.dumps(expected, indent=2))

## 9. Model Loading and Inference (GPU Required)

**Note**: The following cells require a GPU. On Colab, enable GPU runtime first.
On CPU-only systems, these cells will work but be slow.

In [ ]:
# Check device and GPU info
import torch

device = get_device()
print(f"Device: {device}")

if device == "cuda":
    gpu_info = get_gpu_memory_info()
    print(f"GPU: {gpu_info['device_name']}")
    print(f"Total VRAM: {gpu_info['total_memory_gb']:.2f} GB")
else:
    print("⚠️ No GPU detected. Inference will be slow.")
    print("   For Colab: Runtime → Change runtime type → GPU")

In [ ]:
# Load the judge model
# This cell downloads and loads Qwen2.5-1.5B-Instruct with 4-bit quantization
# Takes ~2-5 minutes on first run (model download)

from src.inference import JudgeModel

# Set seed for reproducibility
set_seed(42)

# Initialize judge with Colab T4 optimized config
judge = JudgeModel(config=get_colab_t4_config())

# Load model (downloads on first run)
print("Loading model (this may take a few minutes on first run)...")
judge.load_model()
print("✓ Model loaded successfully!")

In [ ]:
# Run inference on all 3 demo examples
print("=" * 70)
print("RUNNING BASELINE INFERENCE ON DEMO EXAMPLES")
print("=" * 70)

results = []
for i, example in enumerate(DEMO_EXAMPLES, 1):
    print(f"\n--- Example {i}: {example['name']} ---")
    print(f"Prompt: {example['user_prompt']}")
    print(f"Response: {example['candidate_response'][:50]}...")
    
    # Run judge
    raw_output, validation = judge.judge(
        user_prompt=example['user_prompt'],
        candidate_response=example['candidate_response'],
        rubric=example['rubric'],
        validate=True,
    )
    
    results.append({
        "example": example,
        "raw_output": raw_output,
        "validation": validation,
    })
    
    # Display result
    print(f"\n[Model Output]:")
    if validation and validation.is_valid:
        print(json.dumps(validation.parsed_output, indent=2))
        print(f"\n✓ Validation: PASSED")
        print(f"  Score: {validation.parsed_output['score']} (expected: {example['expected_score']})")
    else:
        print(raw_output[:500])
        print(f"\n✗ Validation: FAILED")
        if validation:
            print(f"  Errors: {validation.errors}")

## 10. Results Summary

In [ ]:
# Summary table of results
print("=" * 70)
print("RESULTS SUMMARY")
print("=" * 70)
print(f"\n{'Example':<20} {'Expected':>10} {'Actual':>10} {'Valid':>10} {'Match':>10}")
print("-" * 70)

for r in results:
    name = r['example']['name']
    expected = r['example']['expected_score']
    validation = r['validation']
    
    if validation and validation.is_valid:
        actual = validation.parsed_output['score']
        valid = "Yes"
        match = "✓" if actual == expected else "✗"
    else:
        actual = "N/A"
        valid = "No"
        match = "✗"
    
    print(f"{name:<20} {expected:>10} {actual:>10} {valid:>10} {match:>10}")

# Count successes
valid_count = sum(1 for r in results if r['validation'] and r['validation'].is_valid)
print(f"\n{valid_count}/{len(results)} outputs passed validation")

## 11. CLI Usage Demonstration

You can also run the judge from the command line:

In [ ]:
# CLI usage examples (run from auto-grader directory)
print("=== CLI Usage Examples ===\n")

print("# Basic usage:")
print('python -m src.inference --prompt "What is 2+2?" --response "5" --rubric "Correctness"')

print("\n# With verbose logging:")
print('python -m src.inference --prompt "Who is Apple CEO?" --response "Elon Musk" --rubric "Factuality" --verbose')

print("\n# Without 4-bit quantization (requires more VRAM):")
print('python -m src.inference --prompt "test" --response "test" --rubric "test" --no-4bit')

print("\n# Pipe JSON output to file:")
print('python -m src.inference --prompt "test" --response "test" --rubric "test" > output.json')

print("\n# Custom model:")
print('python -m src.inference --prompt "test" --response "test" --rubric "test" --model "Qwen/Qwen2.5-0.5B-Instruct"')

In [ ]:
# Demonstrate CLI from notebook (optional - uncomment to run)
# This runs the CLI as a subprocess

# import subprocess
# result = subprocess.run(
#     ["python", "-m", "src.inference", 
#      "--prompt", "What is 2+2?",
#      "--response", "5",
#      "--rubric", "Correctness: Answer must be accurate"],
#     capture_output=True, text=True, cwd=project_root
# )
# print("STDOUT (JSON):")
# print(result.stdout)
# print("\nSTDERR (Validation):")
# print(result.stderr)

## 12. Next Steps

This baseline demonstrates the core functionality. Next steps for the project:

1. **Collect Training Data**: Generate diverse (prompt, response, rubric, judgment) pairs
2. **Fine-tune the Model**: Use LoRA/QLoRA for efficient fine-tuning on T4
3. **Evaluate Performance**: Create benchmarks with known-good judgments
4. **Improve Robustness**: Test and harden against prompt injection attempts
5. **Deploy**: Package for inference API

---

**Project Structure Recap:**
```
auto-grader/
├── src/
│   ├── config.py          # Typed configurations
│   ├── prompt_templates.py # Prompt builder with anti-injection
│   ├── io_schema.py       # JSON validation
│   ├── inference.py       # Model loading + CLI
│   └── utils.py           # Seeds, logging
├── tests/                 # Unit tests
└── notebooks/             # This notebook
```